In [1]:
#6. Formulacion cinetica de lagrange

In [2]:
from sympy import symbols, sin, cos
from sympy.physics.mechanics import ReferenceFrame, inertia, Point, dynamicsymbols
from sympy.physics.mechanics import RigidBody, LagrangesMethod, Lagrangian
import numpy as np

In [3]:
# Para el ejemplo del movimiento de un cilindro sobre una rampa
t,m,g,I,R,theta,=symbols('t,m,g,I,R,theta')
q1,q2=dynamicsymbols('q1,q2')

a=ReferenceFrame('A')
d=a.orientnew('D','Axis',(theta,a.z))
c=d.orientnew('C','Axis',(q2,d.z))

#Defina puntos de interés y velocidades
o=Point('O')
c_com=o.locatenew('C_com',d.x*q1)
o.set_vel(a,0)
c_com.set_vel(c,0)

Ic=inertia(c,0,0,I)

#Defina un cuerpo rígido para el cilindro
body_cilindro=RigidBody('cilindro',c_com,c,m,(Ic,c_com))


V=m*g*q1*sin(theta)
T=body_cilindro.kinetic_energy(a)

In [4]:
#Calcule el Lagrangiano
L=T-V

In [5]:
# Utilice el método de Lagrange para obtener las equaciones de movimiento
lm=LagrangesMethod(L,[q1,q2],nonhol_coneqs=[q1.diff(t)-q2.diff(t)*R])
#Observe que se define lambda1 (multiplicador de Lagrange)
lm.form_lagranges_equations()

Matrix([
[g*m*sin(theta) + m*Derivative(q1(t), (t, 2)) + lam1(t)],
[               I*Derivative(q2(t), (t, 2)) - R*lam1(t)]])

In [6]:
# Observe las equaciones de movimiento descritas en la forma MM(q,q')[q' q'' lambda]=[F(q,q') constraints]
MM=lm.mass_matrix_full
F=lm.forcing_full
# Defina el vector xdot como [q' q'' lambda] (expandiendo con lambda)
xdot=lm.q.col_join(lm.u).diff(t)
xdot=xdot.col_join(lm.lam_vec)
MM*(xdot)-F # ==0

Matrix([
[                                                       0],
[                                                       0],
[  g*m*sin(theta) + m*Derivative(q1(t), (t, 2)) - lam1(t)],
[                 I*Derivative(q2(t), (t, 2)) + R*lam1(t)],
[-R*Derivative(q2(t), (t, 2)) + Derivative(q1(t), (t, 2))]])

In [7]:
# Para el ejemplo del modelo de un tren de aterrizaje

# Defina las variables de movimiento
# y parametros del modelo
q1,q2=dynamicsymbols('q1,q2')
# Parámetros del modelo
h,d,mb,mc,Ia,Ir,g=symbols('h,d,mb,mc,Ia,Ir,g')

#Marcos de referencia
a=ReferenceFrame('A')
b=a.orientnew('B','Axis',(q1,a.z))
c=b.orientnew('C','Axis',(q2,b.x))

# Puntos de interes
o=Point('O')
b_com=o.locatenew('Bcom',-h*b.y/2)
b_end=o.locatenew('Bend',-h*b.y)
c_com=b_end.locatenew('Ccom',d*b.x)

# Defina la velocidad de cada punto
o.set_vel(a,0)
b_com.set_vel(b,0)
c_com.set_vel(c,0)
b_com.v2pt_theory(o,a,b)
c_com.v2pt_theory(b_end,b,c)

#Propiedades inerciales
ic=inertia(c, Ia, Ir, Ir)

# Cuerpos rígidos
body_llanta=RigidBody('llanta',c_com,c,mc,(ic,c_com))

# En el ejemplo anterior calculo el Lagrangiano definiendo explícitamente L=T-V
# en este caso utilice el método Lagrangian
body_llanta.potential_energy=mc*g*(h-h*cos(q1)+d*sin(q1))
L=Lagrangian(a,body_llanta).simplify()
L

Ia*Derivative(q2(t), t)**2/2 + Ir*Derivative(q1(t), t)**2/2 - g*mc*(d*sin(q1(t)) - h*cos(q1(t)) + h) + mc*(d**2 + h**2)*Derivative(q1(t), t)**2/2

In [8]:
# Defina el escalar M3 como el momento externo sobre el marco B
M3=symbols('M3')
lm=LagrangesMethod(L,[q1,q2],forcelist=[(b,M3*b.z)],frame=a)
lm.form_lagranges_equations()

Matrix([
[Ir*Derivative(q1(t), (t, 2)) - M3 + g*mc*(d*cos(q1(t)) + h*sin(q1(t))) + mc*(d**2 + h**2)*Derivative(q1(t), (t, 2))],
[                                                                                       Ia*Derivative(q2(t), (t, 2))]])